_

In [ ]:
#| default_exp gstreamer.test_valve

# Gstreamer Valve 
> Gstreamer video capture with on/off valve
> This has been superseded by gstreamer.python




https://github.com/jackersson/gst-python-tutorials
https://github.com/jackersson


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
from UAV.imports import *   # TODO why is this relative import on nbdev_export?
from fastcore.utils import *
import gi
import numpy as np
import threading
gi.require_version('Gst', '1.0')
from gi.repository import Gst
import subprocess
import platform

import paho.mqtt.client as mqtt_client

import time

from pathlib import Path
import logging
import UAV.params as params
from UAV.gstreamer.valve import *

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *


### Default parameters
Overide these default parameters for application specific applications.

The code is shown below:

In [ ]:
show_doc(DefaultParams)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/gstreamer/valve.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### DefaultParams

>      DefaultParams ()

As an example for camera 0, ```DefaultParams.cameras["CAM-0"]["gst"]``` is a list of gstreamer setup commands.

The default parameters list four gst videotestsrc each with a different pattern. 
The patterns are: 
- smpte,  ball, snow, pinwheel.

The video is split with a tee into two streams 
1.  is streamed via an on / off valve to  udp port 5000.
2. is streamed to an appsink for processing.


In [ ]:
gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
print(gstcommand)

['videotestsrc pattern=smpte is-live=true ! tee name=t ', 't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! ', '   videoconvert ! appsink name=sink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ', 't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000']


### Gstreamer Receive Pipeline Class

the class is called with the gst command list and the address and port.
```frame_available``` is a flag that is set when a new frame is available.

In [ ]:
show_doc(GstStream)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/gstreamer/valve.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### GstStream

>      GstStream (name:str='CAM-0', gstcommand:List=['videotestsrc !
>                 autovideosink'], address:str='127.0.0.1', port:int=5000)

"GstStream  class using gstreamer
Create and start a GStreamer pipe
    gst_pipe = GstStream() 
    The valve is a simple element that drops buffers when the drop property is set to TRUE and lets then through otherwise.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str | CAM-0 | camera name |
| gstcommand | List | ['videotestsrc ! autovideosink'] | gst command list |
| address | str | 127.0.0.1 | udp address |
| port | int | 5000 | udp port |

In [ ]:
from UAV.utils.display import show_image
DEFAULT_PIPELINE = "videotestsrc num-buffers=10 ! autovideosink"
gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
with GstStream("CAM-0", [DEFAULT_PIPELINE]) as gststream:
    # gststream.pipeline.set_state(-1000)/
    gststream.pipeline.set_state(Gst.State.PLAYING)
    avail = gststream.frame_available()
    print(f"frame is avail = {avail}")
    # or 
    test_eq(gststream.frame_available(), True)
    ax = show_image(gststream.frame(), figsize=(3,3), rgb2bgr=True)

WARNIN | uav_log         | 32.227 |   valve.py:133 | MainThread         | GstStream Error: appsink is None
INFO   | uav_log         | 32.229 |   valve.py:116 | MainThread         | GstStream started


frame is avail = False


INFO   | uav_log         | 36.287 |   valve.py:269 | MainThread         | GstStream  closed


AssertionError: ==:
False
True

To run the above pipeline 
```gst_pipeline = GstStream()```

To close pipeline, run 
```gst_pipeline.pipeline.close()```

In [ ]:
show_doc(GstStream.frame_available)

In [ ]:
#|eval: true
from UAV.utils.display import show_image
gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
with GstStream("CAM-0", gstcommand) as gststream:
    avail = gststream.frame_available()
    print(f"frame is avail = {avail}")
    # or 
    test_eq(gststream.frame_available(), True)
    ax = show_image(gststream.frame(), figsize=(3,3), rgb2bgr=True)

#### Valve gives the ability to pause the video stream
The valve is a simple element that drops buffers when the drop property is set to TRUE and lets then through otherwise. 

In [ ]:
show_doc(GstStream.set_valve_state)

In [ ]:
show_doc(GstStream.get_valve_state)

Test the valve

In [ ]:
#|eval: true
gstcommand = DefaultParams().cameras["CAM-0"]["gst"]
with GstStream("CAM-0", gstcommand) as gststream:
    avail = gststream.frame_available()
    print(f"frame is avail = {avail}")
    gststream.set_valve_state("myvalve", True)
    test_eq(gststream.get_valve_state("myvalve"), True) 
    gststream.set_valve_state("myvalve", False)
    test_eq(gststream.get_valve_state("myvalve"), False)



#### Ping IP address

In [ ]:
#|eval: false
show_doc(ping_ip)

Test ping IP

In [ ]:
test_eq(ping_ip("127.0.0.1"), True)
test_eq(ping_ip("1.2.3.4"), False)

### MQTT
MQTT is used to control the valve state.
MQTT is a lightweight publish-subscribe messaging protocol that is used on top of TCP/IP. It needs a broker to work. The broker is responsible for distributing messages to interested clients based on the topic of a message. The broker is also responsible for authenticating clients and authorizing them to publish and subscribe to various topics. The broker is the central hub for all communications in the system.
#### install MQTT broker 
```sh
sudo apt-add-repository ppa:mosquitto-dev/mosquitto-ppa.
sudo apt-get update.
sudo apt-get install mosquitto.
sudo apt-get install mosquitto-clients.
sudo apt clean.
```

#### mosquitto-clients
is a set of command line tools that can be used for publishing and subscribing messages to MQTT broker.
For example from second terminal run the following to publish a message to the topic STREAM-CAMERA. This will be received by the mqtt client and the valve state will be set accordingly.
```sh
  mosquitto_pub -m "CAM-0" -t "STREAM-CAMERA"
  mosquitto_pub -m "CAM-1" -t "STREAM-CAMERA"
```

In [ ]:
#|eval: false
show_doc(Mqtt)

In [ ]:
#|eval: false
show_doc(Mqtt.wait_connection)

In [ ]:
#|eval: false
show_doc(Mqtt.on_mqtt_message)


In [ ]:
#|eval: false     don't run this cell in testing as the mqtt broker may not be running
# Test Mqtt
with Mqtt("CAM-0", None) as mqtt:
    mqtt.wait_connection()
    mqtt.client.publish("STREAM-CAMERA", "CAM-0")
    time.sleep(0.1)
    assert mqtt.msg == "CAM-0"

In [ ]:
#|eval: false     don't run this cell in testing as the mqtt broker may not be running
# Test with GstStream

params = DefaultParams()
gstcommand = params.cameras["CAM-0"]["gst"]
with  GstStream("CAM-0", gstcommand) as video, Mqtt("CAM-0", video) as mqtt:

    mqtt.wait_connection()   # wait for connection
    mqtt.client.publish("STREAM-CAMERA", "CAM-0")
    time.sleep(0.1)
    vs = video.get_valve_state("myvalve")
    print(vs)
    test_eq(vs, False)    # ie dont drop frames on this camera, drop on others

    mqtt.client.publish("STREAM-CAMERA", "CAM-1")
    time.sleep(0.1)
    vs = video.get_valve_state("myvalve")
    print(vs)
    test_eq(vs, True)   # ie do drop frames on this camera, don't drop on CAM-1


### Main function for local testing

In [ ]:
#  # |exports
from  UAV.gstreamer.valve import DefaultParams, GstStream, logger, Mqtt
import cv2, time
from imutils import resize
camera = "CAM-0"
gstcommand = DefaultParams().cameras[camera]["gst"]
with  GstStream(camera, gstcommand) as video, Mqtt("CAM-0", video) as mqtt:
    if not mqtt.wait_connection():
        logger.info(' mqtt broker not available - exiting')
        mqtt.close()
        gststream.close()
        sys.exit()   

    if not video.frame_available():
        logger.info('  GST_Frame not available - exiting')
        video.close()
        sys.exit()
        
    cv2.namedWindow(camera, cv2.WINDOW_NORMAL)

    logger.info("""\nSuccess!
        Starting streaming - running 200 frames
        - press 'v' to toggle valve state 
        - press 'q' to quit.""")

    wait_time = 1
    count = 0
    for i in range(2000):

        if video.frame_available() and count % 10 == 0:
            frame = video.frame().copy()
            frame = resize(frame, width= 600)
            cv2.imshow(camera, frame)

        if count % 1000 == 0:
            print( count)
        count += 1

        k = cv2.waitKey(wait_time)

        if k == ord('q') or k == ord('Q') or k == 27:
            break

        if k == ord('v'):
            # Assuming you have a valve element named 'myvalve' in your pipeline
            valve = video.pipeline.get_by_name("myvalve")
            current_drop_state = valve.get_property("drop")
            print(f"current_drop_state {current_drop_state}")
            valve.set_property("drop", not current_drop_state)
            current_drop_state = valve.get_property("drop")
            print(f"new_drop_state {current_drop_state}", )

            time.sleep(2)

        if k == ord(' '):
            if wait_time != 0:
                wait_time = 0
            else:
                wait_time = 1

        if k == ord('s'):
            save = 0
            save_path = Path(params.save_path) 
            save_path.mkdir(exist_ok=True)
            pass
    logger.info("Stopping, 200 frames done")

cv2.destroyAllWindows()

# gst_main("CAM-0")    


#### Receive on UDP with this Test  :
from first terminal run 
```sh
gst-launch-1.0 udpsrc port=5000 ! application/x-rtp,encoding-name=H264,payload=96 ! \
       rtph264depay ! h264parse ! queue ! avdec_h264 ! xvimagesink sync=false async=false -e
```
from second terminal run 
```sh
mosquitto_pub -m "CAM-0" -t "STREAM-CAMERA"
mosquitto_pub -m "CAM-1" -t "STREAM-CAMERA"
```

#### Example: Test with two cameras from terminal
The idea is to run the 4 gst pipelines in different processes

```
from multiprocessing import Process   # you will need to import Process from multiprocessing

if __name__ == '__main__':

    cams = []
    params = DefaultParams()
    for cam in list(params.cameras.keys())[:2]:
        logger.info("Starting Cam: {cam}")
        p = Process(target=main, args=(cam,))
        p.start()
        cams.append(p)

    for p in cams:
        p.join()
```

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()